In [ ]:
import pandas as pd
import os
from glob import glob
import re
from collections import defaultdict

# This code block used for creating combined flight parquet files
# You should have monthly raw files in csv_folder to execute that code block 
# Path to your CSV files
csv_folder = 'raw/'
output_dir = 'raw/Combined_Flights'  # folder to save yearly files
csv_files = glob(os.path.join(csv_folder, '*.csv'))
# Dictionary to collect DataFrames by year
yearly_data = defaultdict(list) 

used_columns = ["Year", "Month", "FlightDate", "Operating_Airline ", "Origin", "Dest", "Flights","DepDelay", "DepDelayMinutes", "ArrDelay", "ArrDelayMinutes", "Diverted"]
# Loop through files and group by year
for file in csv_files:
    print(f"Reading {file}...")
    match = re.search(r'Flights_(\d{4})_', file)
    if match:
        year = match.group(1)
        df = pd.read_csv(file, usecols=used_columns, low_memory=False)
        df.drop(index=df[df['Diverted'] == 1.00].index, inplace=True)
        yearly_data[year].append(df)

# Combine and write yearly files
for year, dfs in yearly_data.items():
    combined_df = pd.concat(dfs, ignore_index=True)
    combined_df.to_parquet(os.path.join(output_dir, f'Combined_Flights_{year}.parquet'), index=False)
    print(f'Written: Combined_Flights_{year}.parquet')

In [1]:
import pandas as pd
from glob import glob
import os

output_dir = 'combined_flights/'
parquet_files = glob(os.path.join(output_dir, '*.parquet'))

# Read combined parquet files and create a dataframe
dfs = []
for file in parquet_files:
    print(f"Reading {file}...")
    dfs.append(pd.read_parquet(file))
df_all = pd.concat(dfs).reset_index(drop=True)
df_all.info()

Reading combined_flights/Combined_Flights_2018.parquet...
Reading combined_flights/Combined_Flights_2019.parquet...
Reading combined_flights/Combined_Flights_2022.parquet...
Reading combined_flights/Combined_Flights_2020.parquet...
Reading combined_flights/Combined_Flights_2021.parquet...
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29125433 entries, 0 to 29125432
Data columns (total 12 columns):
 #   Column              Dtype  
---  ------              -----  
 0   Year                int64  
 1   Month               int64  
 2   FlightDate          object 
 3   Operating_Airline   object 
 4   Origin              object 
 5   Dest                object 
 6   DepDelay            float64
 7   DepDelayMinutes     float64
 8   ArrDelay            float64
 9   ArrDelayMinutes     float64
 10  Diverted            float64
 11  Flights             float64
dtypes: float64(6), int64(2), object(4)
memory usage: 2.6+ GB


In [2]:
# Diverted flights are not direct and has no delay values
df_all.drop("Diverted", axis=1, inplace=True)
df_all.columns

Index(['Year', 'Month', 'FlightDate', 'Operating_Airline ', 'Origin', 'Dest',
       'DepDelay', 'DepDelayMinutes', 'ArrDelay', 'ArrDelayMinutes',
       'Flights'],
      dtype='object')

In [3]:
df_all.columns = df_all.columns.str.strip()
#convert flightdate to datetime
df_all["FlightDate"] = pd.to_datetime(df_all["FlightDate"])
columns = ["FlightDate", "Operating_Airline", "Origin", "Dest", "Flights","DepDelay", "ArrDelay"]
df_all.dropna(axis="index", subset=columns, inplace=True)
df_all.shape

(28346524, 11)

In [4]:
# Merge airline csv instead of codes get full airline name 
airline_map = pd.read_csv("combined_flights/Airlines.csv")
airline_map.columns = airline_map.columns.str.strip()
df_all = df_all.merge(airline_map, how='left', left_on='Operating_Airline', right_on='Code')
df_all.rename(columns={"Description":"Airline"}, inplace=True)
df_all.drop(columns=["Code", 'Operating_Airline'], inplace=True)
df_all.head()

,Year,Month,FlightDate,Origin,Dest,DepDelay,DepDelayMinutes,ArrDelay,ArrDelayMinutes,Flights,Airline
0,2018,8,2018-08-07,SFO,FLL,120.0,120.0,128.0,128.0,1.0,JetBlue Airways
1,2018,8,2018-08-08,SFO,FLL,38.0,38.0,54.0,54.0,1.0,JetBlue Airways
2,2018,8,2018-08-09,SFO,FLL,-1.0,0.0,-1.0,0.0,1.0,JetBlue Airways
3,2018,8,2018-08-10,SFO,FLL,157.0,157.0,146.0,146.0,1.0,JetBlue Airways
4,2018,8,2018-08-11,SFO,FLL,3.0,3.0,3.0,3.0,1.0,JetBlue Airways


In [5]:
len(df_all["Airline"].unique())

28

In [6]:
# most busy routes
df_all["Route"] = df_all["Origin"]+ '-' + df_all["Dest"]
df_new = df_all.groupby("Route").size().reset_index(name='TotalFlights')
df_new.sort_values(by='TotalFlights', ascending=False).head(10)

,Route,TotalFlights
7256,SFO-LAX,54515
4145,LAX-SFO,54468
4094,LAX-LAS,43868
3967,LAS-LAX,43765
4282,LGA-ORD,42249
5580,ORD-LGA,42163
3162,HNL-OGG,40191
5346,OGG-HNL,40147
3813,JFK-LAX,39565
4092,LAX-JFK,39561


In [7]:
len(df_new["Route"].value_counts())

8187

In [ ]:
# 5 airports are selected from the busy routes so that user can make a selection
# This code block is used for generating graphs as json format you can find them under graphs folder
# It generates monthly graphs 1.Flight Traffic by airline  2.Departure Average Delay 3.Arrival Average Delay
import plotly.express as px

def write_to_file(filename, fig):
    with open(f"graphs/{filename}", "w") as f:
        f.write(fig.to_json())


def flight_traffic(org, dest, month, daily_counts):
    # Example: flight traffic per month for multiple years
    fig = px.bar(daily_counts, x='FlightDate', y='TotalFlights', color='Airline', title=org+"-"+dest+' Daily Flight Traffic by Airline-'+str(month))

    # templates  ['ggplot2', 'seaborn', 'simple_white', 'plotly','plotly_white', 'plotly_dark', 'presentation', 'xgridoff','ygridoff', 'gridon', 'none']
    fig.update_layout(
        template='seaborn',
        title_font_size=20,
        xaxis_title='Day of Month',
        yaxis_title='Number of Flights',
        legend_title='Airlines'
    )
    write_to_file((org+"_"+dest+"_"+str(month)+"_traffic.json"), fig)
    #fig.show()

def departure_delay(org, dest, month, dep_delay):
    fig = px.line(dep_delay, x='FlightDate', y='AverageDepDelay', color='Airline', title=org+"-"+dest+' Average Delays per Airline on Departure-'+str(month))

    fig.update_layout(
        template='seaborn',
        title_font_size=20,
        xaxis_title='Day of Month',
        yaxis_title='Delay in minutes',
        legend_title='Airlines'
    )
    #fig.show()
    
    write_to_file((org+"_"+dest+"_"+str(month)+"_departure_delay.json"), fig)

def arrival_delay(org, dest, month, arr_delay):
    fig = px.line(arr_delay, x='FlightDate', y='AverageArrDelay', color='Airline', title=org+"-"+dest+' Average Delays per Airline on Arrival-'+str(month))

    fig.update_layout(
        template='seaborn',
        title_font_size=20,
        xaxis_title='Day of Month',
        yaxis_title='Delay in minutes',
        legend_title='Airlines'
    )
    #fig.show()
    
    write_to_file((org+"_"+dest+"_"+str(month)+"_arrival_delay.json"), fig)

# Through 5 busy routes 
airports = ["SFO", "LAX", "LAS", "ORD", "LGA"]

for org_airport in airports:
    for dest_airport in airports:
        if org_airport == dest_airport:
            continue
        for month in range(1,13):
            # Flight Traffic
            df_filt = ((df_all["Origin"] == org_airport) & (df_all["Dest"] == dest_airport) & (df_all["Month"] == month))
            df_month = df_all[df_filt]
            daily_counts = df_month.groupby([df_month["FlightDate"].dt.day, "Airline"]).size().reset_index(name="TotalFlights")
            daily_counts.sort_values(by=["FlightDate", "TotalFlights"], ascending=[True, False], inplace=True)
            flight_traffic(org_airport, dest_airport, month, daily_counts)

            # Departure Delay
            dep_delay = df_month.groupby([df_month["FlightDate"].dt.day, "Airline"])["DepDelay"].mean().reset_index(name="AverageDepDelay")
            departure_delay(org_airport, dest_airport, month, dep_delay)

            # Arrival Delay
            arr_delay = df_month.groupby([df_month["FlightDate"].dt.day, "Airline"])["ArrDelay"].mean().reset_index(name="AverageArrDelay")
            arrival_delay(org_airport, dest_airport, month, arr_delay)

In [32]:
# Top 10 busy airports
arrivals = df_all.groupby("Dest")["Flights"].sum().rename('Arrivals')
departures = df_all.groupby("Origin")["Flights"].sum().rename('Departures')
airport_traffic = pd.concat([departures, arrivals], axis=1).astype(int)
airport_traffic["Total_Flights"] = airport_traffic["Departures"] + airport_traffic["Arrivals"]
airport_traffic = airport_traffic.nlargest(10, "Total_Flights")
airport_traffic

,Departures,Arrivals,Total_Flights
ATL,1347021,1347297,2694318
ORD,1327423,1325511,2652934
DEN,1137846,1136193,2274039
DFW,1064843,1062170,2127013
CLT,880449,879096,1759545
LAX,833505,834181,1667686
SEA,711511,710776,1422287
IAH,672533,671574,1344107
PHX,661851,661392,1323243
LAS,648466,648924,1297390


In [65]:
import plotly.express as px
from IPython.display import IFrame


airports_dict = {"ATL":"Hartsfield-Jackson Int", "ORD":"O'Hare International", "DEN":"Denver Intl.", "DFW":"Dallas/Ft Worth Intl","CLT":"Douglas Intl", 
        "LAX":"Los Angeles Intl.", "SEA":"Seattle-Tacoma Intl", "IAH":"George Bush Intercont.", "PHX":"Sky Harbor Intl", "LAS":"HARRY REID INTL" }

fig = px.bar(airport_traffic, y=airport_traffic.index, x='Total_Flights', color=airport_traffic.index, title='Top 10 busiest Airports')

fig.update_layout(
    template='plotly',
    title_font_size=20,
    yaxis_title='Airports',
    xaxis_title='Number of Flights',
    legend_title='Airports',
    width=1000
)
fig.update_yaxes(labelalias=airports_dict)
fig.show()

fig.write_html("Top10_busiest_Airports.html")
# Display it inline in notebook
IFrame(src="Top10_busiest_Airports.html", width=1000, height=700)

In [52]:
# average departure delay per airport within top 10 busy airport
airports_index = airport_traffic.index
filt = df_all["Origin"].isin(airports_index)
airport_departure_delay = df_all[filt].groupby("Origin")["DepDelayMinutes"].mean().rename('AvgDepartureDelay')
airport_departure_delay = airport_departure_delay.sort_values(ascending=False).reset_index()
airport_departure_delay

,Origin,AvgDepartureDelay
0,DEN,14.676783
1,ORD,14.383072
2,DFW,14.252769
3,LAS,12.990878
4,IAH,12.932197
5,PHX,11.705703
6,LAX,11.382535
7,CLT,10.990814
8,ATL,9.855629
9,SEA,9.150925


In [81]:
fig = px.bar(airport_departure_delay, y="Origin", x="AvgDepartureDelay", color="Origin", title='Average Departure Delay per Airport')

fig.update_layout(
    template='plotly',
    title_font_size=20,
    yaxis_title='Airports',
    xaxis_title='Average Departure Delay in minutes',
    legend_title='Airports',
    width=1000
)

fig.update_yaxes(labelalias=airports_dict)
fig.show()

fig.write_html("Airports_AvgDeparture_Delay.html")
# Display it inline in notebook
IFrame(src="Airports_AvgDeparture_Delay.html", width=1000, height=700)

In [72]:
# average arrival delay per airport within top 10 busy airport(as destination)
filt = df_all["Dest"].isin(airports_index)
airport_arrival_delay = df_all[filt].groupby("Dest")["ArrDelayMinutes"].mean().rename('AvgArrivalDelay')
airport_arrival_delay = airport_arrival_delay.sort_values(ascending=False).reset_index()
airport_arrival_delay

,Dest,AvgArrivalDelay
0,ORD,16.299875
1,DFW,15.631684
2,IAH,13.524446
3,DEN,13.220039
4,LAX,11.668492
5,LAS,11.650307
6,PHX,11.567681
7,CLT,11.389704
8,SEA,10.080494
9,ATL,9.794389


In [80]:
fig = px.bar(airport_arrival_delay, y="Dest", x="AvgArrivalDelay", color="Dest", title='Average Arrival Delay per Airport')

fig.update_layout(
    template='plotly',
    title_font_size=20,
    yaxis_title='Airports',
    xaxis_title='Average Arrival Delay in minutes',
    legend_title='Airports',
    width=1000
)

fig.update_yaxes(labelalias=airports_dict)
fig.show()

fig.write_html("Airports_AvgArrival_Delay.html")
# Display it inline in notebook
IFrame(src="Airports_AvgArrival_Delay.html", width=1000, height=700)

In [86]:
# Top 10 airlines
df_airlines = df_all.groupby("Airline")["Flights"].count()
df_airlines = df_airlines.nlargest(10)
df_airlines

Airline
Southwest Airlines Co.    5292591
Delta Air Lines Inc.      3242141
SkyWest Airlines Inc.     3077298
American Airlines Inc.    3031275
United Air Lines Inc.     2301368
Republic Airlines         1239828
JetBlue Airways           1073460
Envoy Air                 1033979
Endeavor Air Inc.          974641
PSA Airlines               922947
Name: Flights, dtype: int64

In [95]:
fig = px.bar(df_airlines, y=df_airlines.index, x='Flights', color=df_airlines.index, title='Top 10 Airlines')

fig.update_layout(
    template='plotly',
    title_font_size=20,
    yaxis_title='Airlines',
    xaxis_title='Number of Flights',
    legend_title='Airlines',
    width=1000
)
fig.show()

fig.write_html("Top10_Airlines.html")
# Display it inline in notebook
IFrame(src="Top10_Airlines.html", width=1000, height=700)

In [92]:
# average departure delay per airline within top 10 airlines
airline_index = df_airlines.index
filt = df_all["Airline"].isin(airline_index)
airline_departure_delay = df_all[filt].groupby("Airline")["DepDelayMinutes"].mean().rename('AvgDepartureDelay')
airline_departure_delay = airline_departure_delay.sort_values(ascending=False).reset_index()
airline_departure_delay

,Airline,AvgDepartureDelay
0,JetBlue Airways,20.112351
1,SkyWest Airlines Inc.,13.788298
2,American Airlines Inc.,13.679153
3,United Air Lines Inc.,13.627923
4,PSA Airlines,12.717297
5,Southwest Airlines Co.,11.839668
6,Republic Airlines,10.745211
7,Envoy Air,10.365356
8,Endeavor Air Inc.,10.162563
9,Delta Air Lines Inc.,9.570551


In [96]:
fig = px.bar(airline_departure_delay, y="Airline", x="AvgDepartureDelay", color="Airline", title='Average Departure Delay per Airline')

fig.update_layout(
    template='plotly',
    title_font_size=20,
    yaxis_title='Airlines',
    xaxis_title='Average Departure Delay',
    legend_title='Airlines',
    width=1000
)
fig.show()

fig.write_html("Airlines_Average_Departure_Delay.html")
# Display it inline in notebook
IFrame(src="Airlines_Average_Departure_Delay.html", width=1000, height=700)

In [98]:
# average arrival delay per airline within top 10 airlines
airline_arrival_delay = df_all[filt].groupby("Airline")["ArrDelayMinutes"].mean().rename('AvgArrivalDelay')
airline_arrival_delay = airline_arrival_delay.sort_values(ascending=False).reset_index()
airline_arrival_delay

,Airline,AvgArrivalDelay
0,JetBlue Airways,20.029909
1,SkyWest Airlines Inc.,14.254700
2,United Air Lines Inc.,13.965649
3,American Airlines Inc.,13.884325
4,PSA Airlines,13.157244
5,Republic Airlines,11.996846
6,Envoy Air,11.768185
7,Endeavor Air Inc.,10.574996
8,Southwest Airlines Co.,10.255327
9,Delta Air Lines Inc.,9.502777


In [99]:
fig = px.bar(airline_arrival_delay, y="Airline", x="AvgArrivalDelay", color="Airline", title='Average Arrival Delay per Airline')

fig.update_layout(
    template='plotly',
    title_font_size=20,
    yaxis_title='Airlines',
    xaxis_title='Average Arrival Delay',
    legend_title='Airlines',
    width=1000
)
fig.show()

fig.write_html("Airlines_Average_Arrival_Delay.html")
# Display it inline in notebook
IFrame(src="Airlines_Average_Arrival_Delay.html", width=1000, height=700)

In [115]:
import calendar 
df_month = df_all.groupby("Month")["Flights"].count()
df_dep_delay = df_all.groupby("Month")["DepDelayMinutes"].mean()
df_arr_delay = df_all.groupby("Month")["ArrDelayMinutes"].mean()
df_monthly_delay = pd.concat([df_dep_delay, df_arr_delay, df_month], axis=1).reset_index()
df_monthly_delay["Month_Name"] = df_monthly_delay["Month"].apply(lambda x: calendar.month_name[x])
df_monthly_delay

,Month,DepDelayMinutes,ArrDelayMinutes,Flights,Month_Name
0,1,11.903585,11.957821,2613247,January
1,2,13.346387,13.722086,2265778,February
2,3,10.767342,10.726871,2633624,March
3,4,11.967087,11.966684,2357394,April
4,5,13.048909,13.010987,2308172,May
5,6,16.870481,16.921770,2369762,June
6,7,16.075729,16.087052,2636175,July
7,8,14.664132,14.972913,2176676,August
8,9,9.596135,9.740429,2176366,September
9,10,10.687620,10.869266,2302554,October


In [118]:
fig = px.bar(df_monthly_delay, x="Month_Name", y='Flights', title='Flight Traffic per Month')

fig.update_layout(
    template='plotly',
    title_font_size=20,
    xaxis_title='Months',
    yaxis_title='Number of Flights',
    width=1000
)

fig.show()
fig.write_html("Monthly_Traffic.html")
# Display it inline in notebook
IFrame(src="Monthly_Traffic.html", width=700, height=600)

In [117]:
# average total(arrival+departure) delay per month
df_monthly_delay["Total_AvgDelay"] = df_monthly_delay["DepDelayMinutes"] + df_monthly_delay["ArrDelayMinutes"]
df_monthly_delay

,Month,DepDelayMinutes,ArrDelayMinutes,Flights,Month_Name,Total_AvgDelay
0,1,11.903585,11.957821,2613247,January,23.861407
1,2,13.346387,13.722086,2265778,February,27.068473
2,3,10.767342,10.726871,2633624,March,21.494213
3,4,11.967087,11.966684,2357394,April,23.933771
4,5,13.048909,13.010987,2308172,May,26.059897
5,6,16.870481,16.921770,2369762,June,33.792250
6,7,16.075729,16.087052,2636175,July,32.162781
7,8,14.664132,14.972913,2176676,August,29.637045
8,9,9.596135,9.740429,2176366,September,19.336563
9,10,10.687620,10.869266,2302554,October,21.556886


In [120]:
fig = px.bar(df_monthly_delay, x="Month_Name", y='Total_AvgDelay', title='Total Average Delay per Month')

fig.update_layout(
    template='plotly',
    title_font_size=20,
    xaxis_title='Months',
    yaxis_title='Average Delay in minutes',
    width=1000
)

fig.show()
fig.write_html("Monthly_Average_Delay.html")
# Display it inline in notebook
IFrame(src="Monthly_Average_Delay.html", width=700, height=600)